***
# <font> Introduction to TensorFlowModel using Fashion MNIST dataset</font>
<p style="margin-left:10%; margin-right:10%;"> <font color=teal> 

***

## Overview:

The `TensorFlowModel` class in Accelerated Data Science (ADS) is designed to allow you to rapidly get a model into production. The `.prepare()` method creates the model artifacts that are needed to deploy a functioning model without you having to configure it or write code. However, it does allow you to customize the `score.py` file as needed. Simulate a call to a deployed model with the `.verify()` method. This method calls the `load_model()` and `predict()` functions in the `score.py` file. Using `.verify()` allows you to debug your `score.py` file without having to deploy a model. The `.save()` method pushes your `TensorFlowModel` and the model artifacts to the model catalog. The `.deploy()` method deploys the model to a REST endpoint for you. Finally, the `.predict()` method allows you to call the endpoint to perform model inference.

These simples steps take your trained TensorFlow model and pushes it to production with just a few lines of code.

In [ ]:
#!pip install -U oracle-ads

In [ ]:
!pip install tensorflow-datasets

These installations are done to view the list of available inbuilt datasets in Tensorflow

In [ ]:
!pip install tensorflow_hub

While importing, in addition to the required imports tensorflow_datasets and tensorflow_hub are also added 

Please select the conda envionment tensorflow28_p38_cpu_v1 before proceeding further. 

In [1]:
import ads
import logging
import os
import pandas as pd
import tempfile
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import warnings

from ads.catalog.model import ModelCatalog
from ads.common.model_metadata import UseCaseType

from ads.dataset.factory import DatasetFactory
from ads.model.framework.tensorflow_model import TensorFlowModel
from shutil import rmtree
ads.set_auth(auth='resource_principal')

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)
warnings.filterwarnings('ignore')

  from ads.common.model_metadata import UseCaseType



In [2]:
ads.hello()



  O  o-o   o-o
 / \ |  \ |
o---o|   O o-o
|   ||  /     |
o   oo-o  o--o

ads v2.8.11
oci v2.114.0
ocifs v1.1.3




The list_builders() method displays a list of available datasets under tensorflow

In [3]:
tfds.list_builders()

2023-10-27 12:01:45.488261: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


INFO:absl:Could not refresh the package index (GCS unavailable): All 10 retry attempts failed. The last failure: Error executing an HTTP request: libcurl code 60 meaning 'SSL peer certificate or SSH remote key was not OK', error details: SSL certificate problem: unable to get local issuer certificate
	 when reading metadata of gs://tfds-data/community-datasets-list.jsonl


['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar100_n',
 'cifar10_1',
 'cifar10_corrupted',
 'cifar10_n',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'conll2002',
 'conll2003',
 'controlled_noisy_web_labels'

<a id='intro'></a>
# Introduction

<a id="intro_dataset"></a>
## Dataset

The [Fashion_MNIST](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist) is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.


Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns. The first column consists of the class labels (see above), and represents the article of clothing. The rest of the columns contain the pixel-values of the associated image.

**Note**: This notebook requires public internet access to download the sample dataset.

<a id='create'></a>
# Create a TensorFlow Model

The next cell creates a `TFModel` class that loads in the MNIST data using a public internet connection. The data is then scaled to be between zero and one. To reduce the computational time needed to generate the model, only the first 10,000 training samples are used.

The TensorFlow model has an input layer of 784 nodes (28x28), a single dense, hidden layer of 128 nodes with a RELU activation function. The output layer consists of 10 dense nodes, one for each of the ten possible digits. A 20% drop-out is used to reduce overfitting.

In [4]:
class TFModel:

    fmnist = tf.keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = fmnist.load_data() # Load data
    x_train, x_test = x_train / 255.0, x_test / 255.0        # Scale between 0 and 1
    x_train, y_train = x_train[:10000], y_train[:10000]      # Reduce training data size

    def training(self):
        model = tf.keras.models.Sequential(
            [
                tf.keras.layers.Flatten(input_shape=(28, 28)),
                tf.keras.layers.Dense(128, activation="relu"),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Dense(10),
            ]
        )
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
        model.fit(self.x_train, self.y_train, epochs=1)

        return model

The next cell builds and trains the model:

In [5]:
model = TFModel().training()

2023-10-27 12:06:09.654672: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 12:06:09.655491: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


313/313 [==============================] - 3s 7ms/step - loss: 0.7780 - accuracy: 0.7228


Once the model has been trained, you use the `.predict()` method to make predictions on a subset of the test dataset. Each prediction outputs ten values because there are ten nodes on the output layer. The node with the highest value is the digit that the model predicts.

In [6]:
model.predict(TFModel().x_test[0:1])

array([[-3.8638165, -4.98483  , -3.4681497, -5.0042253, -2.9476   ,
         1.5224715, -3.386241 ,  1.6353451, -0.4474761,  3.4194736]],
      dtype=float32)

<a id='serialize'></a>
# TensorFlow Framework Serialization

The TensorFlow framework makes it easy to deploy a TensorFlow model into production. The `TensorFlowModel()` constructor takes a TensorFlow model and converts it into a `TensorFlowModel` object. To deploy the model into production, you must prepare the model artifact, verify that the artifact works, save the model to the model catalog, and then deploy it.

ADS provides a number of methods that greatly simplify the model deployment process. It also provides the `.summary_status()` method that provides a dataframe that defines the steps, status, and detailed information about each step. 

<a id='serialize_tensorflowmodel'></a>
## Create a TensorFlowModel

In this notebook, a `keras.engine.sequential.Sequential` object is created. The `TensorFlowModel()` constructor takes the `keras.engine.sequential.Sequential` object along with the path that you want to use to store the model artifacts. A `TensorFlowModel` object is returned, and it's used to manage the deployment.

The next cell creates a model artifact directory. This directory is used to store the artifacts that are needed to deploy the model. It also creates the `TensorFlowModel` object.

In [7]:
artifact_dir = tempfile.mkdtemp()
print(f"Model artifact director: {artifact_dir}")
tf_model = TensorFlowModel(estimator=model, artifact_dir=artifact_dir)

Model artifact director: /tmp/tmphp0ijmyi


The `.summary_status()` method of the `TensorFlowModel` class is a handy method to keep track of the progress that you are making in deploying the model. It creates a dataframe that lists the deployment steps, their status, and details about them. The next cell returns the summary status dataframe. It shows that the initiate step has been completed.

In [8]:
tf_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

<a id='serialize_prepare'></a>
## Prepare

The prepare step is performed by the `.prepare()` method of the `TensorFlowModel` class. It creates a number of customized files that are used to run the model once it is deployed. These include:

* `input_schema.json`: A JSON file that defines the nature of the features of the `X_sample` data. It includes metadata such as the data type, name, constraints, summary statistics, feature type, and more.
* `model.h5`: This is the default filename of the serialized model. You can change it with the `model_file_name` attribute. By default, the model is stored in an h5 file. You can use the `as_onnx` parameter to save the model in the ONNX format.
* `output_schema.json`: A JSON file that defines the nature of the dependent variable in the `y_sample` data. It includes metadata such as the data type, name, constraints, summary statistics, feature type, and more.
* `runtime.yaml`: This file contains information that is needed to set up the runtime environment on the deployment server. It has information about which conda environment was used to train the model, and what environment should be used to deploy the model. The file also specifies what version of Python should be used.
* `score.py`: This script contains the `load_model()` and `predict()` functions. The `load_model()` function understands the format the model file was saved in, and loads it into memory. The `.predict()` method is used to make inferences in a deployed model. There are also hooks that allow you to perform operations before and after inference. You are able to modify this script to fit your specific needs.

To create the model artifacts, you use the `.prepare()` method. There are a number of parameters that allow you to store model provenance information. In the next cell, the `conda_env` variable defines the slug of the conda environment that was used to train the model, and the conda environment to use for deployment. Note that you can only pass in slugs to `inference_conda_env` or `training_conda_env` if it's a service environment. Otherwise, you must pass in the full path of the conda environment along with the python version through `inference_python_version` and `training_python_version`.


In [9]:
#conda_env = "generalml_p38_cpu_v1"

tf_model.prepare(
    inference_conda_env="tensorflow28_p38_cpu_v1" ,
    training_conda_env="tensorflow28_p38_cpu_v1",
    use_case_type=UseCaseType.MULTINOMIAL_CLASSIFICATION,
    X_sample=TFModel().x_test,
    y_sample=TFModel().y_test,
)

algorithm: Sequential
artifact_dir:
  /tmp/tmphp0ijmyi:
  - - .model-ignore
    - score.py
    - runtime.yaml
    - output_schema.json
    - model.h5
framework: tensorflow
model_deployment_id: null
model_id: null

The next cell uses the `.summary_status()` method to show you that the prepare step finished, and what tasks were completed:

In [10]:
tf_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

The `.prepare()` method has created the following fully functional files. However, you can modify them to fit your specific needs.

In [11]:
os.listdir(artifact_dir)

['.model-ignore', 'score.py', 'runtime.yaml', 'output_schema.json', 'model.h5']

Once the artifacts have been created, there are a number of attributes in the `TensorFlowModel` object that provide metadata about the model. The `.runtime` attribute details the model deployment settings and model provenance data.

In [12]:
tf_model.runtime_info


model_artifact_version: '3.0'
model_deployment:
  inference_conda_env:
    inference_env_path: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/TensorFlow_2.8_for_CPU_on_Python_3.8/1.0/tensorflow28_p38_cpu_v1
    inference_env_slug: tensorflow28_p38_cpu_v1
    inference_env_type: data_science
    inference_python_version: '3.8'
model_provenance:
  project_ocid: ''
  tenancy_ocid: ''
  training_code:
    artifact_directory: /tmp/tmphp0ijmyi
  training_compartment_ocid: ''
  training_conda_env:
    training_env_path: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/TensorFlow_2.8_for_CPU_on_Python_3.8/1.0/tensorflow28_p38_cpu_v1
    training_env_slug: tensorflow28_p38_cpu_v1
    training_env_type: data_science
    training_python_version: '3.8'
  training_region: ''
  training_resource_ocid: ''
  user_ocid: ''
  vm_image_internal_id: ''

The `.schema_input` attribute provides metadata on the features that were used to train the model. You can use this information to determine what data must be provided to make model inferences. Each feature in the model has a section that defines the dtype, feature type, name, and if it is required. The metadata also includes the summary statistics associated with the feature type.

In [13]:
tf_model.schema_input

schema: []
version: '1.1'

The `.metadata_custom` attribute provides custom metadata that contains information on the category of the metadata, description, key, and value:

In [14]:
tf_model.metadata_custom


data:
- category: Training Environment
  description: The slug name of the training conda environment.
  key: SlugName
  value: tensorflow28_p38_cpu_v1
- category: Other
  description: ''
  key: ClientLibrary
  value: ADS
- category: Training Environment
  description: The URI of the training conda environment.
  key: CondaEnvironmentPath
  value: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/TensorFlow_2.8_for_CPU_on_Python_3.8/1.0/tensorflow28_p38_cpu_v1
- category: Training Environment
  description: The list of files located in artifacts folder.
  key: ModelArtifacts
  value: .model-ignore, score.py, runtime.yaml, model.h5
- category: Other
  description: The model file name.
  key: ModelFileName
  value: model.h5
- category: Training Environment
  description: The conda environment type, can be published or datascience.
  key: EnvironmentType
  value: data_science
- category: Training Profile
  description: The model serialization format.
  key: ModelSerializationFormat


The `.metadata_provenance` contains information about the code and training data that was used to create the model. This information is most useful when a Git repository is being used to manage the code for training the model. This is considered a best practice because it allows you to do things like reproduce a model, perform forensic on the model, and so on.

In [15]:
tf_model.metadata_provenance


artifact_dir: null
git_branch: null
git_commit: null
repository_url: null
training_id: null
training_script_path: null

The `.metadata_taxonomy` is a key-value store that has information about the classification or taxonomy of the model. This can include information such as the model framework, use case type, hyperparameters, and more.

In [16]:
tf_model.metadata_taxonomy

data:
- key: UseCaseType
  value: multinomial_classification
- key: Framework
  value: tensorflow
- key: ArtifactTestResults
  value: null
- key: Hyperparameters
  value: null
- key: FrameworkVersion
  value: 2.8.2
- key: Algorithm
  value: Sequential

<a id='serialize_verify'></a>
## Verify

If you modify the `score.py` file that is part of the model artifacts, then you should verify it. The verify step allows you to test those changes without having to deploy the model. This allows you to debug your code without having to save the model to the model catalog, and then deploy it. The `.verify()` method takes a set of test parameters and performs the prediction by calling the `predict` function in `score.py`. It also runs the `load_model` function.

The next cell simulates a call to a deployed model without having to actually deploy the model. It passes in test values and returns the predictions.

In [17]:
tf_model.verify(TFModel().x_test[0:1])

Start loading model.h5 from model directory /tmp/tmphp0ijmyi ...
Model is successfully loaded.


{'prediction': [[-3.863816499710083,
   -4.984829902648926,
   -3.4681506156921387,
   -5.004225254058838,
   -2.9476001262664795,
   1.5224714279174805,
   -3.3862407207489014,
   1.6353453397750854,
   -0.447476327419281,
   3.419473171234131]]}

Update the `.summary_status()` method to show that the verify step has been completed:

In [18]:
tf_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

<a id='serialize_save'></a>
## Save

Once you are satisfied with the performance of the model and have verified that the `score.py` file is working, you save the model to the model catalog. You do this with the `.save()` method on a `TensorFlowModel` object. This bundles up the model artifact that you have created, and deploys it to the model catalog. It returns the model OCID.

In [19]:
model_id = tf_model.save(display_name='Demo TensorFlowModel model')

Start loading model.h5 from model directory /tmp/tmphp0ijmyi ...
Model is successfully loaded.
['.model-ignore', 'score.py', 'runtime.yaml', 'output_schema.json', 'model.h5']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

<a id='serialize_deploy'></a>
## Deploy

When the model is in the model catalog, you can use the `.deploy()` method of a `TensorFlowModel` object to deploy the model. This method allows you to specify the attributes of the deployment such as the display name, description, instance type and count, the maximum bandwidth, and logging groups. The next cell deploys the model with the default settings, except for the custom display name. The `.deploy()` method returns a `ModelDeployment` object.

In [20]:
deploy = tf_model.deploy(display_name='Demo TensorFlowModel deployment')

loop1:   0%|          | 0/6 [00:00<?, ?it/s]

After deployment, the `.summary_status()` method shows that the model is `ACTIVE` and the `predict()` method is available.

In [21]:
tf_model.summary_status()

Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

<a id='serialize_predict'></a>
## Predict

In the <a href='#create'>Create a TensorFlow Model</a> section, you used the `model.predict()` method where `model` is an `ADSModel` object. This did the inference using the local model. Now that the `TensorFlowModel` model has been deployed, you can do the same thing using similar syntax with the `.predict()` method on a `TensorFlowModel`.

After the deployment is active, you can call `predict()` on the `TensorFlowModel` object to send request to the deployed endpoint. 

In [22]:
tf_model.predict(TFModel().x_test[0:1])['prediction']

[[-3.863816499710083,
  -4.984829902648926,
  -3.4681496620178223,
  -5.004225254058838,
  -2.9475998878479004,
  1.52247154712677,
  -3.3862409591674805,
  1.6353451013565063,
  -0.4474760890007019,
  3.419473648071289]]

<a id='clean_up'></a>
# Clean Up

This notebook created a model deployment and a model. This section deletes those resources. 

The model deployment must be deleted before the model can be deleted. You use the `.delete_deployment()` method on the `TensorFlowModel` object to do this.

In [23]:
delete = tf_model.delete_deployment(wait_for_completion=True)

loop1:   0%|          | 0/2 [00:00<?, ?it/s]

After the model deployment has been deleted, the `.summary_status()` method shows that the model has been deleted and that the `predict()` method is not available:

In [24]:
tf_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  DELETED       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

Use the `.delete_model()` method in a `ModelCatalog` object to delete the model:

In [25]:
ModelCatalog(compartment_id=os.environ['NB_SESSION_COMPARTMENT_OCID']).delete_model(model_id)

True

The next cell removes the model artifacts that were stored on your local drive:

In [26]:
rmtree(artifact_dir)

<a id='ref'></a>
# References
- [ADS Library Documentation](https://docs.cloud.oracle.com/en-us/iaas/tools/ads-sdk/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)
- [Understanding Conda Environments](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)
- [Use Resource Manager to Configure Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/orm-configure-tenancy.htm)
- [`runtime.yaml`](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_runtime_yaml.htm#model_runtime_yaml)
- [`score.py`](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_score_py.htm#model_score_py)
- [Model artifact](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/models_saving_catalog.htm#create-models)
- [ONNX API Summary](http://onnx.ai/sklearn-onnx/api_summary.html)